In [18]:
import sys
import json
sys.path.append('..')
from common import *
from analysis.model_performances import *
from questions_construction.questions import *
from copy import deepcopy
from helpers import *
import pandas as pd



In [19]:
questions_by_id = {d[OUT_OBJ_ID]: d for d in open_jsonl(f'{DATA_PATH}/test_data.paraphrased.cleaned.jsonl')}
override = True

answer_type = TRUE_FALSE_ANSWER_TYPE
answer_response_type = f'{answer_type}.{ACCURACY_SCORE_KEY}' #f'{TRUE_FALSE_ANSWER_TYPE}.{ACCURACY_SCORE_KEY}' #
stats_save_dir = f'{STATISTICS_PATH}.trial_run.ED'


substitution = WITHOUT_RANDOM_SUB
ramification = WITHOUT_RAMIFICATIONS
prompt_type = ZERO_SHOT_PROMPT_KEY #FEW_SHOT_3_PROMPT_KEY #
models = ['gpt-4o', 'llama_8b','llama_70b', 'llama_8b.finetuned_tf']

data_all = []
for model_name in models:
    model_results_dir = f'{PROJECT_PATH}/data/prompting_results/{ramification}/{prompt_type}/{model_name}.jsonl'
    model_results = open_jsonl(model_results_dir)
    data_all_inst = data_all_single_run(questions_by_id, model_results, substitution, ramification, model_name,prompt_type)
    data_all.extend(data_all_inst)

In [20]:
# data_all[0]

In [53]:
def to_df_by_domains(data_all, prompt_type, model, answer_type,
                     ramifications=WITHOUT_RAMIFICATIONS, 
                     subs=WITHOUT_RANDOM_SUB,
                      score_type=ACCURACY_SCORE_KEY,
                      plan_length=19):

    # index = []
    data_for_df = []   
    for plan_length in [1, 10, 19]:
        for question_category in QUESTION_CATEGORIES+[ALL_QUESTION_CATEGORIES_KEY]:
            data_columns = {'plan_length': plan_length, 'question category': question_category}
            for domain in ['blocksworld', 'depots', 'driverlog', 'grippers', 'mystery', 'satellite', 'spanner', 'visitall']:
                stats = TrueFalseStats(data_all, plan_length, question_category, ramifications, model, prompt_type, domain, subs, score_type=score_type)
                res_obj = stats.compute()
                if res_obj:
                    mean = res_obj['result']
                    sem = None
                    if res_obj['result_other']:
                        sem = res_obj['result_other']['sem']
                    final_res = (mean, sem)
                else:
                    final_res = (None, None)
                final_res = tuple([round(v*100, 2) if v else v for v in final_res ])
                final_res = '${'+str(final_res[0])+'}_{'+str(final_res[1])+'}$'
                data_columns[domain] = final_res
            data_for_df.append(data_columns)
    return pd.DataFrame(data_for_df)

# Subs and Ramfications

In [54]:
model = models[0]
df = to_df_by_domains(data_all, prompt_type, model, answer_type)
df
        
# caption_nl = f'performance of {model_name}, {prompt_type}, pl-{plan_length}'.replace('_', ' ')
# save_key = f'{model_name}.{prompt_type}.{plan_length}'
# 
# latex_table_all = to_latex_table(df, caption_nl, label=save_key, index=False)
# with open(os.path.join(save_dir, f'{save_key}.tex'), 'w') as f:
#     f.write(latex_table_all)

,plan_length,question category,blocksworld,depots,driverlog,grippers,mystery,satellite,spanner,visitall
0,1,fluent_tracking,${100.0}_{0.0}$,${80.0}_{17.89}$,${60.0}_{21.91}$,${100.0}_{0.0}$,${100.0}_{0.0}$,${91.67}_{7.98}$,${100.0}_{0.0}$,${100.0}_{0.0}$
1,1,state_tracking,${100.0}_{0.0}$,${88.89}_{10.48}$,${60.0}_{21.91}$,${90.0}_{9.49}$,${88.89}_{10.48}$,${71.43}_{17.07}$,${75.0}_{15.31}$,${100.0}_{0.0}$
2,1,action_executability,${93.33}_{6.44}$,${100.0}_{0.0}$,${100.0}_{0.0}$,${100.0}_{0.0}$,${78.57}_{10.97}$,${100.0}_{0.0}$,${100.0}_{0.0}$,${72.73}_{13.43}$
3,1,effects,${77.78}_{9.8}$,${69.23}_{12.8}$,${83.33}_{15.21}$,${77.78}_{9.8}$,${100.0}_{0.0}$,${77.78}_{13.86}$,${57.14}_{18.7}$,${85.71}_{9.35}$
4,1,numerical_reasoning,${80.0}_{12.65}$,${77.78}_{13.86}$,${66.67}_{15.71}$,${66.67}_{15.71}$,${62.5}_{17.12}$,${57.14}_{18.7}$,${54.55}_{15.01}$,${46.67}_{12.88}$
5,1,composite,${80.0}_{10.33}$,${78.57}_{10.97}$,${76.92}_{11.69}$,${57.14}_{13.23}$,${90.91}_{8.67}$,${70.59}_{11.05}$,${68.75}_{11.59}$,${78.57}_{10.97}$
6,1,ALL_CATEGORIES,${87.34}_{3.74}$,${82.54}_{4.78}$,${77.08}_{6.07}$,${82.93}_{4.16}$,${85.71}_{4.68}$,${79.69}_{5.03}$,${77.27}_{5.16}$,${75.38}_{5.34}$
7,10,fluent_tracking,${100.0}_{0.0}$,${85.71}_{13.23}$,${100.0}_{0.0}$,${100.0}_{0.0}$,${100.0}_{0.0}$,${81.82}_{11.63}$,${81.82}_{11.63}$,${90.91}_{8.67}$
8,10,state_tracking,${88.89}_{10.48}$,${60.0}_{21.91}$,${75.0}_{15.31}$,${80.0}_{12.65}$,${50.0}_{20.41}$,${77.78}_{13.86}$,${75.0}_{15.31}$,${100.0}_{0.0}$
9,10,action_executability,${71.43}_{12.07}$,${75.0}_{12.5}$,${84.62}_{10.01}$,${69.23}_{12.8}$,${58.33}_{14.23}$,${66.67}_{13.61}$,${69.23}_{12.8}$,${71.43}_{12.07}$


In [55]:
latex_table_all = to_latex_table(df, 'some caption', index=False)
print(latex_table_all)


\begin{table}[h!]
\begin{adjustbox}{width=\textwidth,center}
\begin{tabular}{rlllllllll}
\toprule
 plan_length &    question category &       blocksworld &            depots &         driverlog &          grippers &           mystery &         satellite &           spanner &          visitall \\
\midrule
           1 &      fluent_tracking &   ${100.0}_{0.0}$ &  ${80.0}_{17.89}$ &  ${60.0}_{21.91}$ &   ${100.0}_{0.0}$ &   ${100.0}_{0.0}$ &  ${91.67}_{7.98}$ &   ${100.0}_{0.0}$ &   ${100.0}_{0.0}$ \\
           1 &       state_tracking &   ${100.0}_{0.0}$ & ${88.89}_{10.48}$ &  ${60.0}_{21.91}$ &   ${90.0}_{9.49}$ & ${88.89}_{10.48}$ & ${71.43}_{17.07}$ &  ${75.0}_{15.31}$ &   ${100.0}_{0.0}$ \\
           1 & action_executability &  ${93.33}_{6.44}$ &   ${100.0}_{0.0}$ &   ${100.0}_{0.0}$ &   ${100.0}_{0.0}$ & ${78.57}_{10.97}$ &   ${100.0}_{0.0}$ &   ${100.0}_{0.0}$ & ${72.73}_{13.43}$ \\
           1 &              effects &   ${77.78}_{9.8}$ &  ${69.23}_{12.8}$ & ${83.33}_{15.21}$ 

/Users/paveldolin/dev/research/completed/reasoning_about_actions/pipeline/src/analysis/tables/helpers.py:45: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex_table = df.to_latex(index=index, formatters={"name": str.upper}, float_format="{:.2f}".format)


In [80]:
df_pos_neg = to_df_by_sub_ram_neg_pos(data_all, model_name, prompt_type, plan_length=plan_length)
df_pos_neg
        
# caption_nl = f'performance of {model_name}, {prompt_type}, pl-{plan_length}'.replace('_', ' ')
# save_key = f'{model_name}.{prompt_type}.{plan_length}'
# 
# latex_table_all = to_latex_table(df, caption_nl, label=save_key, index=False)
# with open(os.path.join(save_dir, f'{save_key}.tex'), 'w') as f:
#     f.write(latex_table_all)

NameError: name 'OUT_OBJ_IS_POS_FLUENT_QUESTION' is not defined

# By Models

In [45]:
score_type=ACCURACY_SCORE_KEY, 
question_category = ALL_QUESTION_CATEGORIES_KEY,
domain = ALL_DOMAINS_KEY,
answer_type=TRUE_FALSE_ANSWER_TYPE,
plan_length=1

data_for_df = []    
for fluent_type in FLUENT_TYPES_LIST:
    # index.append()
    data_columns = {}
    data_columns['fluent type'] = TO_PRETTY[fluent_type]
    for model in models:
        data = filter_multi_selector_modified(data_all, ramification, model, prompt_type, answer_type, substitution, plan_length, [(OUT_OBJ_FLUENT_TYPE, {fluent_type})])
        print(len(data))
        stats = TrueFalseStatsCustom(data, plan_length, question_category, ramification, model, prompt_type, domain, substitution, score_type=score_type)
        res_obj = stats.compute()
        if res_obj:
            mean = res_obj['result']
            sem = None
            if res_obj['result_other']:
                sem = res_obj['result_other']['sem']
            final_res = (mean, sem)
        else:
            final_res = (None, None)
        final_res = tuple([round(v*100, 2) if v else v for v in final_res ])
        final_res = '${'+str(final_res[0])+'}_{'+str(final_res[1])+'}$'
        # print(final_res)
        data_columns[TO_PRETTY.get(model, model)] = final_res
    data_for_df.append(data_columns)

0
0
0
0
0
0
0
0
0
0
0
0


In [64]:
data = filter_multi_selector_modified(data_all, ramification, 'llama_8b', prompt_type, 'free_answer', substitution, 1, [(OUT_OBJ_FLUENT_TYPE, {fluent_type})])

In [65]:
data

[]

In [29]:
prompt_type

'zero_shot'

In [54]:
prompt_type

'zero_shot'

In [55]:
substitution

'without_random_sub'

In [56]:
data_all[0]

{'question_id': '784689cc-3f7c-4a22-824a-ffc0e4c90483',
 'domain_name': 'logistics',
 'instance_id': 'Instance_2',
 'question_category': 'composite',
 'question_name': 'iter_6_question_2',
 'fluent_type': 'all_fluents',
 'answer_type': 'free_answer',
 'question': 'Given the initial condition, the following actions are planned to be performed: at airport l1_1, package p2 is loaded in truck t1, at airport l1_1, package p1 is loaded in truck t1, truck t1 is driven to airport l1_0 from airport l1_1 in city c1, in city c1, truck t0 is driven from airports l1_1 to l0_1, from truck t1 package p1 is unloaded at airport l1_0, in city c0, truck t0 is driven from airports l0_1 to l0_0, at airport l0_0, package p3 is loaded in airplane a0, airplane a0 flies from airports l0_0 to l1_0, from airplane a0 package p3 is unloaded at airport l1_0 and package p3 is loaded in truck t1 at airport l1_0 to reach the current state. What are the valid properties of the state that do not involve negations for p1